In [6]:
import os
import torchaudio
import torch
import glob
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from dataset import MultiDataset
from sklearn.model_selection import KFold

In [5]:
# Load metadata CSV file 
path = "C:/Users/annammc/Documents/Gestures"
audiofile_list = glob.glob(os.path.join(path, "*.wav"))
videofile_list = glob.glob(os.path.join(path, "*.mov"))
midifile_list = glob.glob(os.path.join(path, "*.mid"))
mocapcsv_list = glob.glob(os.path.join(path, "*.csv"))


In [11]:
# Convert the audio features to numpy 
audio_features_list = []
for audio_file in audiofile_list:
    features = MultiDataset(audio_file)
    audio_features_list.append(features)

# Split the audio features into chunks
chunk_size = 1000
audio_chunks = [features[i:i+chunk_size] for i in range(0, len(features), chunk_size)]

# Save the audio chunks as individual npy files
for i, chunk in enumerate(audio_chunks):
    np.save(f"/itf-fi-ml/shared/users/annammc/audio_features/audio_chunk{i+1}.npy", chunk)
        

In [12]:
db_gesture = "C:/Users/annammc/Downloads/Anna/test/*"
files = glob.glob(db_gesture)
filename = [os.path.basename(item) for item in files]

target = [item.split("_")[-2] for item in files]

label_encoder = LabelEncoder()
target_idx = label_encoder.fit_transform(target) # Change class names to numeric

dict = {'audio_npy': filename, 'target': target, 'target_idx': target_idx}
dataset_all = pd.DataFrame(dict)

dataset_all.to_csv('metadata.csv')


In [14]:
X = dataset_all['audio_npy']
y = dataset_all['target_idx']

# Define the number of folds
num_folds = 5

# Perform K-Fold cross-validation
kf = KFold(n_splits=num_folds, shuffle=True)

fold = 0
for train_indices, test_indices in kf.split(X):
    print(f"Fold {fold + 1}/{num_folds}")
    
    # Split the data for this fold
    X_train, X_test = X[train_indices], X[test_indices]
    y_train, y_test = y[train_indices], y[test_indices]
    
    train_df = pd.DataFrame({'filename': list(X_train), 'target': list(y_train)})
    test_df = pd.DataFrame({'filename':list(X_test), 'target': list(y_test)})
    
    train_df.to_csv(f"train_fold{fold+1}.csv")
    test_df.to_csv(f"test_fold{fold+1}.csv")
    
    fold += 1